In [3]:
%matplotlib inline
import pandas as pd
import numpy as np


# Agrupaciones de datos

Las agrupaciones son operaciones necesarias para analizar datos, ya que permiten extraer información en función de datos categóricos de nuestro dataframe.

Cargaremos los datos llamados _experiment.csv_ que podemos encontrar en el siguiente [enlace](https://raw.githubusercontent.com/bmalcover/MADM2019/master/data/experiment.csv)

In [6]:
df= pd.read_csv("data/experiment.csv")
df

,Nombre,Apellidos,Altura,Sexo,Nacimiento,Cof,Categoria
0,Will,Smith,1.43,M,10/10/1920,0.19,laboral
1,Jon,Snow,1.98,M,10/1/1970,0.98,laboral
2,Laia,Ramirez,1.87,F,09/10/1987,0.76,cap6
3,Luzy,Raim,1.67,F,23/07/1979,0.56,cap6
4,Fein,Mang,1.78,M,12/03/1937,0.27,cap6
5,Victor,Colom,1.78,M,22/09/1957,0.97,cap8


En el siguiente ejemplo agrupamos los datos según el sexo de la persona mediante el método `groupby` que devuelve un `DataFrame` agrupado:

In [9]:
bySex = df.groupby('Sexo')
type(bySex)

pandas.core.groupby.generic.DataFrameGroupBy

El atributo `groups` nos muestra los grupos hemos creado:

In [10]:
# Podemos saber los grupos realizados y que índices del dataframe tienen.
bySex.groups # nos proporciona un diccionario


{'F': [2, 3], 'M': [0, 1, 4, 5]}

Esto nos permite realizar operaciones de filtrado con base a los grupos que hemos creado:

In [45]:
df.loc[bySex.groups['M'].values]

,Nombre,Apellidos,Altura,Sexo,Nacimiento,Cof,Categoria
0,Will,Smith,1.43,M,10/10/1920,0.19,laboral
1,Jon,Snow,1.98,M,10/1/1970,0.98,laboral
4,Fein,Mang,1.78,M,12/03/1937,0.27,cap6
5,Victor,Colom,1.78,M,22/09/1957,0.97,cap8


### Funciones de agregación en grupos.

Con los datos agrupados podemos realizar operaciones para obtener información de cada categoria.

In [9]:
bySex.describe()

Altura                                                     Cof          \
      count    mean       std   min     25%   50%   75%   max count    mean   
Sexo                                                                          
F       2.0  1.7700  0.141421  1.67  1.7200  1.77  1.82  1.87   2.0  0.6600   
M       4.0  1.7425  0.228674  1.43  1.6925  1.78  1.83  1.98   4.0  0.6025   

                                                
           std   min   25%   50%     75%   max  
Sexo                                            
F     0.141421  0.56  0.61  0.66  0.7100  0.76  
M     0.431383  0.19  0.25  0.62  0.9725  0.98

Para ejemplificar esta sección, agruparemos el _dataframe_ por `Categoria` laboral. En este caso para la columna `Altura` consultamos la suma de las alturas del grupo y `Cof` la media.

La función `aggregate` nos permite crear variables de agregación sobre la tabla obtenida con `groupby`. Indicaremos la información que queremos obtener de cada columna con un diccionario. Especificamos la función que vamos a aplicar a los datos de cada grupo en cada columna para obtener un único valor.

In [6]:
df.groupby(["Categoria"]).aggregate({
    "Altura":np.sum,
    "Cof":np.mean})

,Cof,Altura
Categoria,,
cap6,0.530,5.32
cap8,0.970,1.78
laboral,0.585,3.41


**Podemos aplicar un gran número de funciones de agregación:**

- [Funciones estadísticas](https://docs.scipy.org/doc/numpy/reference/routines.statistics.html): mean, std, ...

- [Funciones matemáticas](https://docs.scipy.org/doc/numpy/reference/routines.math.html): sum, prod, ...

- Otras funciones: max, min, ...

- [Documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html)


### Agrupaciones de múltiples columnas

También se pueden realizar agrupaciones de múltiples columnas. Se crean todas las combinaciones de las diversas columnas que existen en el DataFrame. Veamos un ejemplo:

In [37]:
gr = df.groupby(['Sexo',"Categoria"]).mean()
print(gr)
gr.index

                Altura    Cof
Sexo Categoria               
F    cap6        1.770  0.660
M    cap6        1.780  0.270
     cap8        1.780  0.970
     laboral     1.705  0.585


MultiIndex([('F',    'cap6'),
            ('M',    'cap6'),
            ('M',    'cap8'),
            ('M', 'laboral')],
           names=['Sexo', 'Categoria'])

Si queremos realizar un conteo de los elementos, debemos seleccionar

In [39]:
gr = df.groupby(['Sexo',"Categoria"])["Sexo"].count()
print(gr)

Sexo  Categoria
F     cap6         2
M     cap6         1
      cap8         1
      laboral      2
Name: Sexo, dtype: int64


### Acceso a los datos de un grupo

Si queremos obtener los datos de una sola categoria, podemos hacerlo mediante el _string_ que representa esta categoría:

In [20]:
for name, group in df.groupby('Categoria'): 
    # Imprime el nombre
    print(name)
    # Imprime el grupo
    print(group)
    
    print("_"*60)

cap6
  Nombre Apellidos  Altura Sexo  Nacimiento   Cof Categoria
2   Laia   Ramirez    1.87    F  09/10/1987  0.76      cap6
3   Luzy      Raim    1.67    F  23/07/1979  0.56      cap6
4   Fein      Mang    1.78    M  12/03/1937  0.27      cap6
____________________________________________________________
cap8
   Nombre Apellidos  Altura Sexo  Nacimiento   Cof Categoria
5  Victor     Colom    1.78    M  22/09/1957  0.97      cap8
____________________________________________________________
laboral
  Nombre Apellidos  Altura Sexo  Nacimiento   Cof Categoria
0   Will     Smith    1.43    M  10/10/1920  0.19   laboral
1    Jon      Snow    1.98    M   10/1/1970  0.98   laboral
____________________________________________________________


Así pues, para consultar los datos de un único grupo se puede hacer usando el método `get_group` de la siguiente manera:

In [10]:
df.groupby('Categoria')

,Nombre,Apellidos,Altura,Sexo,Nacimiento,Cof,Categoria
2,Laia,Ramirez,1.87,F,09/10/1987,0.76,cap6
3,Luzy,Raim,1.67,F,23/07/1979,0.56,cap6
4,Fein,Mang,1.78,M,12/03/1937,0.27,cap6


También puede ser interesante la utilitzación del método `groups` para conocer como están formados nuestros grupos.

# Ejercicios

**1) Usando el fichero WHO.csv, ¿Cuál es el volumen total de CO2 emitido por cada continente?**

**2) ¿Cuál es el número de paises por continente?**

**3) Carga el fichero climaMallorca.csv: ¿Cual es la temperatura máxima cuando el viento es inferior a 10? ¿Cuántas muestras hay?**

**4) ¿Cual es la temperatura máxima cuando el viento es superior a 10 y inferior a 20? ¿Cuántas muestras hay?**

**5) Del conjunto "who.csv" selecciona al azar: 30 paises y calcula la media de "Net primary school enrolment ratio female (%)" agrupado por Continente**

Nota: la selección de 30 paises se puede hacer con una función del objeto DataFrame

**6) Repite la anterior actividad pero ahora con todos los paises. ¿Sale la misma media?**

**7) Repasemos: Guarda el fichero WHO.csv en un fichero HDF5**